In [13]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
random.seed(108)

In [14]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [15]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"Joint_Displacement(JD,4)": 0,
                           "Crack_surface_damage(S,3)": 1,
                           "Sludge_Accumulation(Sl,3)": 2,
                           "Stone(St,3)": 3,
                           "Blockage(B,4)": 4,
                           "Fracture_surface_damage_(S,3)": 5
                           }

In [16]:
# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join(info_dict["filename"].replace("jpg", "txt"))
    #save_file_name = os.path.join("train", info_dict["filename"].replace("jpg", "txt"))

    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))
    
# for x in os.listdir('E:/test - all/label_val'):
#   print(x)

In [17]:
for x in os.listdir('C:/Users/YOGESH V/Desktop/BlockageAI/road_sign_data/fracture_SD/labels'):
    print(x)

fracture_surface_damage_00000.xml
fracture_surface_damage_00001.xml
fracture_surface_damage_00002.xml
fracture_surface_damage_00003.xml
fracture_surface_damage_00004.xml
fracture_surface_damage_00005.xml
fracture_surface_damage_00006.xml
fracture_surface_damage_00007.xml
fracture_surface_damage_00008.xml
fracture_surface_damage_00009.xml
fracture_surface_damage_00010.xml
fracture_surface_damage_00011.xml
fracture_surface_damage_00012.xml
fracture_surface_damage_00013.xml
fracture_surface_damage_00014.xml
fracture_surface_damage_00015.xml
fracture_surface_damage_00016.xml
fracture_surface_damage_00017.xml
fracture_surface_damage_00018.xml
fracture_surface_damage_00019.xml
fracture_surface_damage_00020.xml
fracture_surface_damage_00021.xml
fracture_surface_damage_00022.xml
fracture_surface_damage_00023.xml
fracture_surface_damage_00024.xml
fracture_surface_damage_00025.xml
fracture_surface_damage_00026.xml
fracture_surface_damage_00027.xml
fracture_surface_damage_00028.xml
fracture_surfa

In [18]:
test = [os.path.join(x) for x in os.listdir('C:/Users/YOGESH V/Desktop/BlockageAI/road_sign_data/fracture_SD/labels') if x[-3:] == "xml"]
test.sort()

for ann in tqdm(test):
    info_dict = extract_info_from_xml('C:/Users/YOGESH V/Desktop/BlockageAI/road_sign_data/fracture_SD/labels/'+ ann)
    convert_to_yolov5(info_dict)

100%|██████████████████████████████████████████████████████████████████████████████| 340/340 [00:00<00:00, 1680.72it/s]

Invalid Class. Must be one from  dict_keys(['Joint_Displacement(JD,4)', 'Crack_surface_damage(S,3)', 'Sludge_Accumulation(Sl,3)', 'Stone(St,3)', 'Blockage(B,4)', 'Fracture_surface_damage_(S,3)'])
Invalid Class. Must be one from  dict_keys(['Joint_Displacement(JD,4)', 'Crack_surface_damage(S,3)', 'Sludge_Accumulation(Sl,3)', 'Stone(St,3)', 'Blockage(B,4)', 'Fracture_surface_damage_(S,3)'])
Invalid Class. Must be one from  dict_keys(['Joint_Displacement(JD,4)', 'Crack_surface_damage(S,3)', 'Sludge_Accumulation(Sl,3)', 'Stone(St,3)', 'Blockage(B,4)', 'Fracture_surface_damage_(S,3)'])
Invalid Class. Must be one from  dict_keys(['Joint_Displacement(JD,4)', 'Crack_surface_damage(S,3)', 'Sludge_Accumulation(Sl,3)', 'Stone(St,3)', 'Blockage(B,4)', 'Fracture_surface_damage_(S,3)'])
Invalid Class. Must be one from  dict_keys(['Joint_Displacement(JD,4)', 'Crack_surface_damage(S,3)', 'Sludge_Accumulation(Sl,3)', 'Stone(St,3)', 'Blockage(B,4)', 'Fracture_surface_damage_(S,3)'])
Invalid Class. Must 